The friedman test

In [4]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import requests
import json
from pandas.io.json import json_normalize

import utils as utils

import sys
from tqdm import tqdm
%pylab inline
plt.style.use("bmh")
np.random.seed(983456)

Populating the interactive namespace from numpy and matplotlib


In [5]:
es = Elasticsearch([{'host': '35.178.209.118', 'use_ssl': False, 'read timeout':20}]) 

In [20]:
def fetch_res_pmids(name,es,index):
    include=["pmid", "last_author_name","last_author.forename","last_author.affiliation"]
    es_query={"query":{"term":{"last_author_name": name }},"_source":{"includes":include},"size":10000} 
    res=es.search(index=index,
                 body=es_query)
    # res=json_normalize(res)
    # return res
    #print(es_query)
    #print(res)
    return pd.DataFrame([h["_source"] for h in res["hits"]["hits"]])

In [21]:
df=fetch_res_pmids('Friedman, N',es,"bulk_papers5")

In [22]:
df.pmid

0     25249765
1     26606974
2     25171404
3     25024161
4     24244020
5     26446933
6     12740579
7     17190847
8     29514105
9     23219532
10    23334501
11    24421210
12    24312094
13    23935451
14    23352245
15    23769506
16    22925037
17    11560351
18    12151303
19    12161879
20    12176835
21    11344753
22    15454407
23    15262806
24    12843144
25    14764868
26    12015876
27    11473012
28    21444810
29    21109421
30    20529911
31    21685068
32    21685097
33    19968953
34    19846608
35    18463706
36    16117504
37    16597232
38    18586706
39    18494234
40    17237082
41    10154065
42    28731407
43    28716917
44    25801168
45    27496019
46    22984176
47    23089682
48    23072772
49    29903938
50    30605682
51    31464369
Name: pmid, dtype: object

In [23]:
def fetch_res_detail(pmid,es,index):
    include=["pmid", "doi","last_author_id","last_author_name","last_author_forename","last_author.affiliation","last_author_email","title","pub_year","last_author_country","last_author_inst",\
    "journal_name","language","authors","mesh","mesh_major"]
    es_query={"query":{"terms":{"pmid": pmid }},"_source":{"includes":include},"size":10000} 
    res=es.search(index=index,
                 body=es_query)
    # res=json_normalize(res)
    # return res
    #print(es_query)
    #print(res)
    return pd.DataFrame([h["_source"] for h in res["hits"]["hits"]])

In [50]:
df_enriched=fetch_res_detail(df.pmid.to_list(),es,'bulk_papers5')
df_enriched.head(3)

,pub_year,last_author,last_author_inst,journal_name,language,pmid,title,last_author_country,last_author_name,authors,last_author_id,mesh_major,mesh,last_author_email,doi
0,2014,{'affiliation': 'Departments of General Medici...,Barwon Health,"Clinical medicine insights. Circulatory, respi...",eng,25249765,Adherence to therapeutic guidelines for patien...,Australia,"Friedman, N","[{'forename': 'Nr', 'affiliation': 'School of ...",14025.0,NaN,NaN,NaN,NaN
1,2016,"{'affiliation': 'Department of Immunology, Wei...",Weizmann Institute of Science,Immunology and cell biology,eng,26606974,Clonal expansion under the microscope: studyin...,Israel,"Friedman, N","[{'forename': 'Michal', 'affiliation': 'Depart...",806421.0,"[Cell Differentiation, Clonal Evolution, Lymph...","[Animals, Cell Communication, Cell Tracking/me...",NaN,NaN
2,2014,"{'affiliation': 'Department of Immunology, Wei...",Weizmann Institute of Science,Cell,eng,25171404,Paradoxical signaling by a secreted molecule l...,Israel,"Friedman, N","[{'forename': 'Yuval', 'affiliation': 'Departm...",806421.0,"[CD4-Positive T-Lymphocytes/cytology, Interleu...","[Animals, Cell Count, Cell Death, Cell Prolife...",[nir.friedman@weizmann.ac.il],10.1016/j.cell.2014.07.033


In [45]:
lst=[]
for pub in df.pmid.to_list():
    r=requests.get('http://s2-public-api-prod.us-west-2.elasticbeanstalk.com/v1/paper/PMID:'+str(pub))
    json=r.json()
    dat=(json['authors'][-1]['authorId'])
    lst.append({'pmid': pub, 'author_id': dat})
lst



[{'pmid': '25249765', 'author_id': '145454371'},
 {'pmid': '26606974', 'author_id': '145618792'},
 {'pmid': '25171404', 'author_id': '145618792'},
 {'pmid': '25024161', 'author_id': '145618792'},
 {'pmid': '24244020', 'author_id': '145618792'},
 {'pmid': '26446933', 'author_id': '50785579'},
 {'pmid': '12740579', 'author_id': '145618792'},
 {'pmid': '17190847', 'author_id': '50785462'},
 {'pmid': '29514105', 'author_id': '145618792'},
 {'pmid': '23219532', 'author_id': '145618792'},
 {'pmid': '23334501', 'author_id': '50785579'},
 {'pmid': '24421210', 'author_id': '50785579'},
 {'pmid': '24312094', 'author_id': '145618792'},
 {'pmid': '23935451', 'author_id': '145618792'},
 {'pmid': '23352245', 'author_id': '145618792'},
 {'pmid': '23769506', 'author_id': '50785484'},
 {'pmid': '22925037', 'author_id': '50785579'},
 {'pmid': '11560351', 'author_id': '50785462'},
 {'pmid': '12151303', 'author_id': '50785579'},
 {'pmid': '12161879', 'author_id': '50785462'},
 {'pmid': '12176835', 'author

In [47]:
df2=pd.DataFrame(lst)
df2



,pmid,author_id
0,25249765,145454371
1,26606974,145618792
2,25171404,145618792
3,25024161,145618792
4,24244020,145618792
5,26446933,50785579
6,12740579,145618792
7,17190847,50785462
8,29514105,145618792
9,23219532,145618792


In [51]:
df_full = pd.merge(df_enriched, df2, on='pmid', how='left')
df_full

,pub_year,last_author,last_author_inst,journal_name,language,pmid,title,last_author_country,last_author_name,authors,last_author_id,mesh_major,mesh,last_author_email,doi,author_id
0,2014,{'affiliation': 'Departments of General Medici...,Barwon Health,"Clinical medicine insights. Circulatory, respi...",eng,25249765,Adherence to therapeutic guidelines for patien...,Australia,"Friedman, N","[{'forename': 'Nr', 'affiliation': 'School of ...",14025.0,NaN,NaN,NaN,NaN,145454371
1,2016,"{'affiliation': 'Department of Immunology, Wei...",Weizmann Institute of Science,Immunology and cell biology,eng,26606974,Clonal expansion under the microscope: studyin...,Israel,"Friedman, N","[{'forename': 'Michal', 'affiliation': 'Depart...",806421.0,"[Cell Differentiation, Clonal Evolution, Lymph...","[Animals, Cell Communication, Cell Tracking/me...",NaN,NaN,145618792
2,2014,"{'affiliation': 'Department of Immunology, Wei...",Weizmann Institute of Science,Cell,eng,25171404,Paradoxical signaling by a secreted molecule l...,Israel,"Friedman, N","[{'forename': 'Yuval', 'affiliation': 'Departm...",806421.0,"[CD4-Positive T-Lymphocytes/cytology, Interleu...","[Animals, Cell Count, Cell Death, Cell Prolife...",[nir.friedman@weizmann.ac.il],10.1016/j.cell.2014.07.033,145618792
3,2014,"{'affiliation': 'Department of Immunology, Wei...",Weizmann Institute of Science,Genome research,eng,25024161,T-cell receptor repertoires share a restricted...,Israel,"Friedman, N","[{'forename': 'Asaf', 'affiliation': 'Departme...",806421.0,"[Nucleotide Motifs, Receptors, Antigen, T-Cell...","[Animals, CD4-Positive T-Lymphocytes/immunolog...","[irun.cohen@weizmann.ac.il, nir.friedman@weizm...",NaN,145618792
4,2013,NaN,Weizmann Institute of Science,"Journal of immunology (Baltimore, Md. : 1950)",eng,24244020,Coupled IL-2-dependent extracellular feedbacks...,Israel,"Friedman, N","[{'forename': 'Nir', 'affiliation': 'Departmen...",806421.0,"[CD4-Positive T-Lymphocytes/immunology, Feedba...","[Amino Acid Sequence, Animals, Antigen Present...",NaN,10.4049/jimmunol.1301575,145618792
5,2015,{'affiliation': 'School of Computer Science & ...,NaN,Molecular systems biology,eng,26446933,Condition-specific genetic interaction maps re...,Israel,"Friedman, N","[{'forename': 'Jenia', 'affiliation': 'School ...",808548.0,"[Cyclic AMP/metabolism, Cyclic AMP-Dependent P...",[Yeasts/enzymology],[nir@cs.huji.ac.il],NaN,50785579
6,2003,NaN,Stanford University,Nature genetics,eng,12740579,Module networks: identifying regulatory module...,United States,"Friedman, N","[{'forename': 'Eran', 'affiliation': 'Computer...",NaN,[Gene Expression Profiling/statistics & numeri...,"[Algorithms, Databases, Genetic, Gene Expressi...",NaN,10.1038/ng1165,145618792
7,2007,NaN,Tel Aviv University,The Annals of pharmacotherapy,eng,17190847,Brand versus generic alendronate: gastrointest...,Israel,"Friedman, N","[{'forename': 'Hillel', 'affiliation': 'Chairm...",NaN,"[Alendronate/adverse effects, Drugs, Generic/a...","[Aged, Chemistry, Pharmaceutical, Cohort Studi...",NaN,10.1345/aph.1H218,50785462
8,2018,{'affiliation': 'School of Computer Science an...,Hebrew University of Jerusalem,Cell reports,eng,29514105,Fine-Resolution Mapping of TF Binding and Chro...,Israel,"Friedman, N","[{'forename': 'Jenia', 'affiliation': 'School ...",NaN,NaN,NaN,[nir.friedman@mail.huji.ac.il],NaN,145618792
9,2013,NaN,Weizmann Institute of Science,Molecular cell,eng,23219532,Dynamic response diversity of NFAT isoforms in...,Israel,"Friedman, N","[{'forename': 'Nissan', 'affiliation': 'Depart...",806421.0,"[Cell Nucleus/metabolism, NFATC Transcription ...","[Animals, Calcium/physiology, Cell Line, Droso...",NaN,10.1016/j.molcel.2012.11.003,145618792


In [52]:
df_full.to_json('friedmans_test.json', orient='records')